In [120]:
__name__ == "__main__"

True

In [121]:
# --- Modèle théorique d'estimation des émissions sur le périphérique parisien ---
# Inspiré de la décomposition de Bigo (2020)

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import voila
from ipywidgets import FloatSlider, IntSlider, interactive, Layout
from IPython.display import display
from copy import deepcopy

In [122]:
# --- Paramètres par défaut ---
def default_params():
    return {
        "distance_moyenne_km": 7,
        "nb_usagers_jour": 1_000_000,
        "part_voitures_essence": 0.7,
        "part_voitures_diesel": 0.2,
        "part_voitures_elec": 0.1,
        "conso_l_100km_essence_70": 7.0,
        "conso_l_100km_essence_50": 6.5,
        "conso_l_100km_diesel_70": 5.5,
        "conso_l_100km_diesel_50": 5.0,
        "conso_kWh_100km_elec_70": 17.0,
        "conso_kWh_100km_elec_50": 15.5,
        "emission_CO2_l_essence": 2.3,
        "emission_CO2_l_diesel": 2.6,
        "emission_CO2_kWh": 0.012,
        "emission_NOx_g_km_essence": 0.25,
        "emission_NOx_g_km_diesel": 0.6,
        "emission_NOx_g_km_elec": 0.01,
        "emission_PM10_g_km_essence": 0.01,
        "emission_PM10_g_km_diesel": 0.05,
        "emission_PM10_g_km_elec": 0.005,
        "taux_remplissage_actuel": 1.2,
        "taux_remplissage_covoit": 1.8,
        "part_covoiturage": 0.2,
        "duree_jours": 365,
        "part_temps_pointe": 0.31,
        "part_poids_lourd": 0.03,
        "jours_semaine": 260,
        "jours_weekend": 90,
        "jours_vacances": 15,
    }

In [123]:
def facteur_congestion(nb_vehicules, seuil, seuil_max, nb_voies):
    """
    Estime un facteur de surconsommation lié à la congestion, en fonction
    de la densité de trafic par voie.

    Le facteur renvoyé est utilisé pour ajuster la consommation énergétique 
    (carburant ou électricité) des véhicules lorsque la densité de trafic 
    augmente.

    Paramètres :
    - nb_vehicules : nombre total de véhicules sur le tronçon considéré
    - seuil : seuil de densité (veh/voie) à partir duquel la consommation commence à augmenter
    - seuil_max : densité à partir de laquelle la congestion est considérée comme forte
    - nb_voies : nombre de voies disponibles pour les véhicules considérés

    Retour :
    - Un facteur multiplicatif appliqué à la consommation (1.0 = trafic fluide, >1.0 = congestion)
    
    Comportement :
    - Si densité ≤ seuil        → consommation normale (facteur = 1.0)
    - Si seuil < densité ≤ max  → surconsommation modérée (croissance linéaire)
    - Si densité > seuil_max    → surconsommation sévère (facteur > 1.1)
    """

    densite = nb_vehicules / nb_voies

    if densite <= seuil:
        return 1.0
    elif densite <= seuil_max:
        return 1.0 + 0.1 * (densite - seuil) / (seuil_max - seuil)
    else:
        return 1.1 + 0.1 * (densite - seuil_max) / seuil_max

In [124]:
def get_vehicule_data(params, type_v, vitesse_reduite):
    if type_v == "essence":
        conso = params["conso_l_100km_essence_50"] if vitesse_reduite else params["conso_l_100km_essence_70"]
        return conso, params["emission_CO2_l_essence"], params["emission_NOx_g_km_essence"], params["emission_PM10_g_km_essence"]
    elif type_v == "diesel":
        conso = params["conso_l_100km_diesel_50"] if vitesse_reduite else params["conso_l_100km_diesel_70"]
        return conso, params["emission_CO2_l_diesel"], params["emission_NOx_g_km_diesel"], params["emission_PM10_g_km_diesel"]
    else:
        conso = params["conso_kWh_100km_elec_50"] if vitesse_reduite else params["conso_kWh_100km_elec_70"]
        return conso, params["emission_CO2_kWh"], params["emission_NOx_g_km_elec"], params["emission_PM10_g_km_elec"]

In [125]:
def capacite_theorique_voie_route(params):
    """
    Calcule la capacité horaire maximale d'une route selon une formule empirique --> https://tunnels.piarc.org/sites/tunnels/files/public/wysiwyg/import/Chapters%20PIARC%20reports/2001%2005.11.B%20Chap%204%20FR.pdf 

    - N : nombre de voies
    - Cl : coefficient de longueur
    - Cpl : coefficient de placement (accès latéraux)
    - Cc : coefficient de conditions

    Retourne :
    - capacité_journalière maximale en véhicules/jour
    """
    Cl = 1.0
    Cc = 1.0
    Cpl = (1 + params["part_poids_lourd"] * 0.5) ** -1
    capacite_horaire = 2200 * Cl * Cpl * Cc                 #on ne multiplie par par le nombre de voie car on le fait plus tard dans le code, pour différencier les scénarios de activation ou non activation de la voie de covoiturage
    capacite_journaliere = capacite_horaire * 24
    return capacite_journaliere

In [126]:
def calcul_seuils_congestion(params, nb_voies):
    """
    Calcule les seuils de congestion (début et max) en fonction de la capacité de la route.

    - nb_voies : nombre de voies considérées
    - params : dictionnaire contenant 'capacite_par_voie'

    Retourne :
    - seuil : début de congestion (trafic dense)
    - seuil_max : congestion sévère
    """
    
    capacite_voie = capacite_theorique_voie_route(params) #en vhc/h/voie
    seuil = capacite_voie * nb_voies
    seuil_max = capacite_voie * 1.5 * nb_voies  #congestion sévère à +50% du trafic possible
    return seuil, seuil_max

In [127]:
def compute_emissions_for_group(params, n_vehicules, congestion, part_temps, vitesse_reduite):
    d = params["distance_moyenne_km"]
    jours = params["duree_jours"]
    result = {"CO2": 0, "NOx": 0, "PM10": 0}

    for type_v, part_motor in [
        ("essence", params["part_voitures_essence"]),
        ("diesel", params["part_voitures_diesel"]),
        ("elec", params["part_voitures_elec"])
    ]:
        conso, CO2e, NOx, PM = get_vehicule_data(params, type_v, vitesse_reduite)
        km_total = part_motor * n_vehicules * d * part_temps
        consommation_totale = km_total * (conso / 100) * congestion

        result["CO2"] += consommation_totale * CO2e / 1000
        result["NOx"] += km_total * NOx / 1_000_000
        result["PM10"] += km_total * PM / 1_000_000

    return result

In [128]:
def calcul_emissions(params, vitesse_reduite=False, part_temps=1.0, voie_covoit_activee=False):
    
    d = params["distance_moyenne_km"]
    #jours = params["duree_jours"]
    nb_usagers_jour = params["nb_usagers_jour"]
    D = nb_usagers_jour * d    #demande en passagers.km par jour

    part_solo = 1 - params["part_covoiturage"]
    r = 1 / (part_solo + params["part_covoiturage"] / params["taux_remplissage_covoit"]) #taux de remplissage 
    n = D / (r * d)                                                                      #nombre moyen d'usager par jour
    
    #print(f"\n>>> Nombre de véhicules ajusté : {n:.0f}")

    result = {"CO2": 0, "NOx": 0, "PM10": 0}

    if voie_covoit_activee and part_temps == params["part_temps_pointe"]:
        #print(">>> Mode spécial : voie covoiturage activée aux heures de pointe")
        part_prioritaire = min(0.05 + params["part_covoiturage"], 1.0)
        part_classique = 1 - part_prioritaire

        if part_prioritaire >= 0.25:
            #print(">>> Trop de véhicules prioritaires : retour à répartition sur 4 voies")
            seuil, seuil_max = calcul_seuils_congestion(params, nb_voies=4)
            congestion = facteur_congestion(n, seuil, seuil_max, nb_voies=4)
            result = compute_emissions_for_group(params, n, congestion, part_temps, vitesse_reduite)
        else:
            n_prio = n * part_prioritaire
            n_class = n * part_classique

            seuil_p, seuil_max_p = calcul_seuils_congestion(params, nb_voies=1)
            seuil_c, seuil_max_c = calcul_seuils_congestion(params, nb_voies=3)

            congestion_p = facteur_congestion(n_prio, seuil_p, seuil_max_p, nb_voies=1)
            congestion_c = facteur_congestion(n_class, seuil_c, seuil_max_c, nb_voies=3)

            print(f"  → part_prioritaire : {part_prioritaire:.4f}")
            print(f"  → densité prio : {n_prio:.0f} / 1 voie")
            print(f"  → densité classique : {n_class:.0f} / 3 voies")
            print(f"  → congestion prio : {congestion_p:.4f}")
            print(f"  → congestion classique : {congestion_c:.4f}")

            res_prio = compute_emissions_for_group(params, n_prio, congestion_p, part_temps, vitesse_reduite)
            res_class = compute_emissions_for_group(params, n_class, congestion_c, part_temps, vitesse_reduite)

            for k in result:
                result[k] = res_prio[k] + res_class[k]
    else:
        seuil, seuil_max = calcul_seuils_congestion(params, nb_voies=4)
        congestion = facteur_congestion(n, seuil, seuil_max, nb_voies=4)
        result = compute_emissions_for_group(params, n, congestion, part_temps, vitesse_reduite)

    # Emissions moyennes par personne
    #nb_personnes_total = nb_usagers_jour * jours
    #result["CO2_par_personne"] = result["CO2"] * 1000 / nb_personnes_total
    #result["NOx_par_personne"] = result["NOx"] * 1_000_000 / nb_personnes_total
    #result["PM10_par_personne"] = result["PM10"] * 1_000_000 / nb_personnes_total

    return result

In [129]:
def emissions_sur_duree(params, nb_jours, vitesse_reduite=False, voie_covoit_activee=False):
    """
    Calcule les émissions totales (CO2, NOx, PM10) pour une période personnalisée
    (ex: 1 jour, 30 jours, 365 jours, etc.), en tenant compte de la répartition
    moyenne des types de jour (semaine, week-end, vacances) et du scénario choisi.
    """
    jours_total = params["jours_semaine"] + params["jours_weekend"] + params["jours_vacances"]
    props = {
        "Semaine": params["jours_semaine"] / jours_total,
        "Week-end": params["jours_weekend"] / jours_total,
        "Vacances": params["jours_vacances"] / jours_total,
    }

    types_de_jour = {
        "Semaine": {"facteur": 1.0},
        "Week-end": {"facteur": 0.78},
        "Vacances": {"facteur": 0.4}
    }

    result = {"CO2": 0, "NOx": 0, "PM10": 0}

    for jour_type, part in props.items():
        local_params = params.copy()
        local_params["nb_usagers_jour"] *= types_de_jour[jour_type]["facteur"]

        part_pointe = params["part_temps_pointe"]
        part_off = 1 - part_pointe

        res_pointe = calcul_emissions(
            local_params, vitesse_reduite=vitesse_reduite, part_temps=part_pointe, voie_covoit_activee=voie_covoit_activee
        )
        res_off = calcul_emissions(
            local_params, vitesse_reduite=vitesse_reduite, part_temps=part_off, voie_covoit_activee=False
        )

        res_total = {k: res_pointe[k] + res_off[k] for k in res_pointe}
        for k in result:
            result[k] += res_total[k] * part * nb_jours

    return result

In [130]:
def emissions_par_personne_sur_duree(params, duree_jours, vitesse_reduite=False, voie_covoit_activee=False):
    emissions = emissions_sur_duree(params, duree_jours, vitesse_reduite, voie_covoit_activee)
    total_personnes = params["nb_usagers_jour"] * duree_jours
    moyennes = {
        "CO2_kg_j": emissions["CO2"] * 1000 / total_personnes,
        "NOx_g_j": emissions["NOx"] * 1_000_000 / total_personnes,
        "PM10_g_j": emissions["PM10"] * 1_000_000 / total_personnes
    }

    return {
        "CO2_total_t": emissions["CO2"],
        "NOx_total_t": emissions["NOx"],
        "PM10_total_t": emissions["PM10"],
        **moyennes
    }

In [131]:
def afficher_resultats(params, duree_jours=1):
    """
    Affiche les émissions totales et par personne pour différents scénarios,
    sur une durée personnalisable (en jours).
    """

    scenarios = {
        "A. 70 km/h sans voie covoiturage": emissions_par_personne_sur_duree(deepcopy(params), duree_jours, vitesse_reduite=False, voie_covoit_activee=False),
        "B. 50 km/h sans voie covoiturage": emissions_par_personne_sur_duree(deepcopy(params), duree_jours, vitesse_reduite=True, voie_covoit_activee=False),
        "C. 50 km/h avec voie covoiturage": emissions_par_personne_sur_duree(deepcopy(params), duree_jours, vitesse_reduite=True, voie_covoit_activee=True)
    }

    base = scenarios["A. 70 km/h sans voie covoiturage"]
    print(f"\nÉmissions totales sur {duree_jours} jours pour {params['nb_usagers_jour']:,} usagers/jour :")

    for nom, valeurs in scenarios.items():
        print(f"\n{nom} :")
        for polluant in ["CO2_total_t", "NOx_total_t", "PM10_total_t"]:
            val = valeurs[polluant]
            if nom == "A. 70 km/h sans voie covoiturage":
                print(f"  {polluant} : {val:.2f}")
            else:
                ref = base[polluant]
                variation = 100 * (val - ref) / ref
                signe = "+" if variation >= 0 else ""
                print(f"  {polluant} : {val:.2f} ({signe}{variation:.1f}%)")

    print("\n--- Émissions moyennes par personne ---")
    for nom, valeurs in scenarios.items():
        print(f"\n{nom} :")
        print(f"  CO2 par jour : {valeurs['CO2_kg_j']:.1f} kg")
        print(f"  NOx par jour : {valeurs['NOx_g_j']:.1f} g")
        print(f"  PM10 par jour : {valeurs['PM10_g_j']:.1f} g")

    # --- Données pour les graphes ---
    labels = list(scenarios.keys())
    x = np.arange(len(labels))
    CO2_vals = [scenarios[k]["CO2_total_t"] for k in labels]
    NOx_vals = [scenarios[k]["NOx_total_t"] for k in labels]
    PM10_vals = [scenarios[k]["PM10_total_t"] for k in labels]

    def intervalle_confiance(valeurs, pourcentage=0.02):
        ecart = np.array(valeurs) * pourcentage
        return np.array(valeurs) - ecart, np.array(valeurs) + ecart

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    co2_bas, co2_haut = intervalle_confiance(CO2_vals)
    axes[0].plot(x, CO2_vals, marker='o', color='green')
    axes[0].fill_between(x, co2_bas, co2_haut, color='green', alpha=0.2)
    axes[0].set_title("Émissions de CO₂")
    axes[0].set_ylabel("t")
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(labels, rotation=20)

    nox_bas, nox_haut = intervalle_confiance(NOx_vals)
    axes[1].plot(x, NOx_vals, marker='o', color='orange')
    axes[1].fill_between(x, nox_bas, nox_haut, color='orange', alpha=0.2)
    axes[1].set_title("Émissions de NOx")
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(labels, rotation=20)

    pm10_bas, pm10_haut = intervalle_confiance(PM10_vals)
    axes[2].plot(x, PM10_vals, marker='o', color='gray')
    axes[2].fill_between(x, pm10_bas, pm10_haut, color='gray', alpha=0.2)
    axes[2].set_title("Émissions de PM10")
    axes[2].set_xticks(x)
    axes[2].set_xticklabels(labels, rotation=20)

    plt.tight_layout()
    plt.show()

In [132]:
if __name__ == "__main__":
    pass

In [133]:
"""
# --- Interface interactive ---
@interact(
    distance_moyenne_km=FloatSlider(min=1, max=20, step=1, value=7, description="km/jour"),
    nb_usagers_jour=IntSlider(min=100_000, max=2_000_000, step=50_000, value=1_000_000, description="Usagers/jour"),
    part_covoiturage=FloatSlider(min=0.0, max=0.5, step=0.01, value=0.2, description="% covoiturage"),
    taux_remplissage_covoit=FloatSlider(min=1.5, max=4, step=0.1, value=1.8, description="Pers/vhc covoit"),
    surconso_congestion_pointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.1, description="Surconso pointe"),
    surconso_congestion_offpointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.0, description="Surconso hors-pointe")
    #seuil_congestion=FloatSlider(min=50000, max=200000, step=5000, value=100000, description="Seuil congestion"),
    #seuil_max_congestion=FloatSlider(min=100000, max=300000, step=5000, value=200000, description="Seuil max congestion")
)
def simulation_interactive(distance_moyenne_km, nb_usagers_jour, part_covoiturage, taux_remplissage_covoit,
                           surconso_congestion_pointe, surconso_congestion_offpointe):
                           #seuil_congestion, seuil_max_congestion):
    params = default_params()
    params.update({
    "distance_moyenne_km": distance_moyenne_km,
    "nb_usagers_jour": nb_usagers_jour,
    "part_covoiturage": part_covoiturage,
    "taux_remplissage_covoit": taux_remplissage_covoit,
    "surconso_congestion_pointe": surconso_congestion_pointe,
    "surconso_congestion_offpointe": surconso_congestion_offpointe,
    #"seuil_congestion": seuil_congestion,
    #"seuil_max_congestion": seuil_max_congestion
})
    afficher_resultats(params)
"""

'\n# --- Interface interactive ---\n@interact(\n    distance_moyenne_km=FloatSlider(min=1, max=20, step=1, value=7, description="km/jour"),\n    nb_usagers_jour=IntSlider(min=100_000, max=2_000_000, step=50_000, value=1_000_000, description="Usagers/jour"),\n    part_covoiturage=FloatSlider(min=0.0, max=0.5, step=0.01, value=0.2, description="% covoiturage"),\n    taux_remplissage_covoit=FloatSlider(min=1.5, max=4, step=0.1, value=1.8, description="Pers/vhc covoit"),\n    surconso_congestion_pointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.1, description="Surconso pointe"),\n    surconso_congestion_offpointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.0, description="Surconso hors-pointe")\n    #seuil_congestion=FloatSlider(min=50000, max=200000, step=5000, value=100000, description="Seuil congestion"),\n    #seuil_max_congestion=FloatSlider(min=100000, max=300000, step=5000, value=200000, description="Seuil max congestion")\n)\ndef simulation_interactive(distance_moyenne_k

In [ ]:
# Layout et style communs
common_style = {'description_width': '150px'}
common_layout = Layout(width='400px')

# Définir les paramètres dans un dictionnaire
param_configs = {
    "distance_moyenne_km": dict(cls=FloatSlider, min=1, max=20, step=1, value=7, description="km/jour"),
    "nb_usagers_jour": dict(cls=IntSlider, min=100_000, max=2_000_000, step=50_000, value=1_000_000, description="Usagers/jour"),
    "part_covoiturage": dict(cls=FloatSlider, min=0.0, max=0.5, step=0.01, value=0.2, description="% covoiturage"),
    "taux_remplissage_covoit": dict(cls=FloatSlider, min=1.5, max=4, step=0.1, value=1.8, description="Pers/vhc covoit"),
    "surconso_congestion_pointe": dict(cls=FloatSlider, min=1.0, max=1.2, step=0.01, value=1.1, description="Surconso pointe"),
    "surconso_congestion_offpointe": dict(cls=FloatSlider, min=1.0, max=1.2, step=0.01, value=1.0, description="Surconso hors-pointe"),
    "part_voitures_essence": dict(cls=FloatSlider, min=0, max=1, step=0.1, value=0.7, description="% vhc essence"),
    "part_voitures_diesel": dict(cls=FloatSlider, min=0, max=1, step=0.1, value=0.2, description="% vhc diesel"),
    "part_voitures_elec": dict(cls=FloatSlider, min=0, max=1, step=0.1, value=0.1, description="% vhc elec")
}

# Générer les sliders automatiquement
sliders = {}
for name, config in param_configs.items():
    sliders[name] = config["cls"](
        min=config["min"],
        max=config["max"],
        step=config["step"],
        value=config["value"],
        description=config["description"],
        style=common_style,
        layout=common_layout
    )

# Fonction interactive
def simulation_interactive(**kwargs):
    params = default_params()
    params.update(kwargs)
    afficher_resultats(params)

# Interface interactive
interactive_widget = interactive(simulation_interactive, **sliders)
display(interactive_widget)

interactive(children=(FloatSlider(value=7.0, description='km/jour', layout=Layout(width='400px'), max=20.0, mi…